In [ ]:
import streamlit as st
from langchain_core.messages import HumanMessage, AIMessage
from langgraph_backend import chatbot
import pyttsx3
import speech_recognition as sr
import threading
import uuid

# Streamlit Page Config
st.set_page_config(page_title="LangGraph Chatbot", page_icon="🤖", layout="centered")

st.title("🤖 LangGraph Chatbot")
st.write("A chatbot built with LangGraph + Streamlit + Voice")

# Initialize TTS engine once
engine = pyttsx3.init()

# Initialize session state
if "messages" not in st.session_state:
    st.session_state["messages"] = []
if "thread_id" not in st.session_state:
    st.session_state["thread_id"] = str(uuid.uuid4())  # unique thread id for checkpointing

# Voice Input Function
def get_voice_input():
    recognizer = sr.Recognizer()
    recognizer.pause_threshold = 1.2
    recognizer.dynamic_energy_threshold = True
    mic = sr.Microphone()
    
    with mic as source:
        st.info("🎤 Listening... Speak now!")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = recognizer.listen(source, timeout=10, phrase_time_limit=30)
        except sr.WaitTimeoutError:
            st.error("⚠️ No speech detected. Try again.")
            return None
    
    try:
        text = recognizer.recognize_google(audio, language="en-US")
        st.success(f"🗣️ You said: {text}")
        return text
    except sr.UnknownValueError:
        st.error("❌ Could not understand the audio.")
        return None
    except sr.RequestError:
        st.error("⚠️ Speech Recognition service error.")
        return None

# Speak Function (non-blocking)
def speak_text(text):
    def run_speech():
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=run_speech, daemon=True).start()

# Chat Function
def chat_with_bot(user_input):
    # Speak user input as well
    speak_text(user_input)

    # Save human message
    st.session_state["messages"].append(HumanMessage(content=user_input))

    # Call LangGraph chatbot with thread_id (fixes invoke error)
    response = chatbot.invoke(
        {"messages": st.session_state["messages"]},
        config={"configurable": {"thread_id": st.session_state["thread_id"]}}
    )

    ai_reply = response["messages"][-1].content
    st.session_state["messages"].append(AIMessage(content=ai_reply))

    # Voice output for bot
    speak_text(ai_reply)

# Input Options
col1, col2 = st.columns(2)

with col1:
    user_text = st.text_input("💬 Type your message:")

with col2:
    if st.button("🎤 Speak"):
        voice_input = get_voice_input()
        if voice_input:
            chat_with_bot(voice_input)

if user_text:
    chat_with_bot(user_text)


# Display chat history
st.subheader("Chat History")
for msg in st.session_state["messages"]:
    if isinstance(msg, HumanMessage):
        st.markdown(f"**🧑 You:** {msg.content}")
    elif isinstance(msg, AIMessage):
        st.markdown(f"**🤖 Bot:** {msg.content}")
